## EVAL.G: evaluation of likelihood function and its derivatives

In [2]:
import numpy as np
import math as math
import cmath

In [5]:
def eval_():
    
    #EVALUATE PROBABILITY OF KEEPING BUS ENGINE AT EACH SAMPLE POINT
    lp = pk[dt[:,1]-1,:]
    
    if i < 0:
        
        #Go to Bypass
        print("Go to Bypass")

        
    #COMPUTE DERIVATIVES OF LOGLIKELIHOOD FUNCTION
    
    #CASE 1: ESTIMATE FULL LIKELIHOOD BUT NOT bet
    
    if (modnum[[2,3]].flatten() == np.array([[2,0]])).all():
            
        d1v = np.concatenate([(-np.ones(nt,1)), dc[dt[:,1]-1,:], np.zeros(nt, modnum[4]) ])
        
    
    #CASE 2: ESTIMATE FULL LIKELIHOOD AND BET
    
    elif (modnum[[2,3]].flatten() == np.array([[2,1]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), (dc[dt[:,1]-1,:]), (-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1,:])),
                              np.zeros((nt,int(modnum[4])))], axis=1)        
        
    
    
    #CASE 3: ESTIMATE PARTIAL LIKELIHOOD BUT NOT bet
    
    elif (modnum[[2,3]].flatten() == np.array([[1,0]])).all():
    
        d1v = np.concatenate([-np.ones((nt,1)), dc[dt[:,1]-1,:]], axis=1)
        
   
    #CASE 4: ESTIMATE PARTIAL LIKELIHOOD AND bet
    
    elif (modnum[[2,3]].flatten() == np.array([[1,1]])).all():
        
        d1v = np.concatenate([-np.ones((nt,1)), dc[dt[:,1]-1,:], (-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1,:]))])
    
    
    #ADD ON DERIVATIVES OF EXPECTED VALUE FUNCTION, BYPASS IF bet=0 
    
    if bet == 0:
        
        d1v = d1v *(lp-1+dt[:,0].reshape(dt.shape[0],1))            
        
        if modnum[2]==2:
            
            g = dm - modnum[4] + 1
            
            d1v[:,(g-1):dm] = invp[dt[:,2],:]
            
        
        print("Go to Cumulate")
        #GOTO CUMULATE
        
   
    #FIRST TWO DERIVATIVES (tr,c1) ARE SAME FOR ALL MODELS
    
    d1v[:,0:2] = (d1v[:,0:2]+dev[0,0:2]*np.ones((nt,1))-dev[:,0:2][dt[:,1]-1,:])*(lp-1+dt[:,0].reshape(dt.shape[0],1))
    
    
    
    #DERIVATIVES FOR SUMMANDS OF PARTIAL LOGLIKELIHOOD FUNCTION
    
    if modnum[2] == 1:
    
        for g in range(3,dm+1):

            u = (np.array([d1v[:,(g-1)] + dev[0,(g-1)]]).T * np.ones((nt,1))-np.array([dev[:,(g-1)]]).T[dt[:,1]-1,:]) * (lp-1+dt[:,0].reshape(dt.shape[0],1))
            d1v[:,(g-1)] = u.flatten()
    
    
    #DERIVATIVES FOR SUMMANDS OF FULL LOGLIKELIHOOD FUNCTION 
    
    elif modnum[2] == 2:
            
         for g in range(3,dm-int(modnum[4])+1):

                u = (np.array([d1v[:,(g-1)] + dev[0,(g-1)]]).T * np.ones((nt,1))-np.array([dev[:,(g-1)]]).T[dt[:,1]-1,:]) * (lp-1+dt[:,0].reshape(dt.shape[0],1))
                d1v[:,(g-1)] = u.flatten()
        
        
        g = dm - int(modnum[4]) + 1
        
        d1v[:,(g-1):dm]=((dev[0,(g-1):dm]*np.ones((nt,1))-dev[:,(g-1):dm][dt[:,1]-1,:]))*(lp-1+dt[:,0].reshape(dt.shape[0],1)) + invp[dt[:,2],:]
        
    
    
   

## CUMULATE

In [301]:
#CUMULATE DERIVATIVES AND LOGLIKELIHOOD VALUES OVER SAMPLE POINTS
#CUMULATE DERIVATIVES OF LOGLIKELIHOOD, dll 

dll = np.array([d1v.sum(axis=0)]).T + dll

## BYPASS

In [417]:
#FULL LOGLIKELIHOOD CUMULATION 

if modnum[2] == 2:
    
    ll =  ll + np.array([np.sum(np.array([list(map(cmath.log,(lp+(1-2*lp)*np.array([dt[:,0]]).T)))]).T +  lnp[dt[:,2],:], axis=0)]).T
    

#PARTIAL LOGLIKELIHOOD CUMULATION 

    ll =  ll + np.array([np.sum(np.array([list(map(cmath.log,(lp+(1-2*lp)*np.array([dt[:,0]]).T)))]).T, axis=0)]).T


In [3]:
dl = np.array([[2,4,6,8,10],[12,1,2,3,14],[16,18,20,22,24]])
dt = np.array([[3,1,1],[4,2,2],[5,3,3]])
dev = np.array([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]])
lp = np.array([[2],[4],[6]])
lnp = np.array([[2,4,6],[8,10,12],[14,16,18],[20,22,24],[26,28,30]])
invp = np.array([[1,2,3],[5,6,7],[9,10,11],[13,14,15],[17,18,19]])
modnum = np.array([[0],[1],[2],[3],[1]])
dll = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15]])
dm = 4
nt = 3
d1v = dl
g = 2

In [250]:
ev = np.array([[1,3,5],[7,9,11],[13,15,17]])
dc = np.array([[2,4,6],[8,10,12],[14,16,18]])
bet = 0.6